In [1]:
import cv2, time, pandas
from datetime import datetime

first_frame = None
status_list = [None, None] 
times = []
df = pandas.DataFrame(columns= ["Start", "End"])

video = cv2.VideoCapture(0)   

time.sleep(2)
while True:
    check, frame = video.read()
    status = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0) 
    if first_frame is None:
        first_frame = gray
        continue  

    delta_frame = cv2.absdiff(first_frame,gray)
    thres_frame = cv2.threshold(delta_frame, 80, 255, cv2.THRESH_BINARY)[1]
    thres_frame = cv2.dilate(thres_frame, None, iterations=2)

    
    cnts,hier = cv2.findContours(thres_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

   
    for contour in cnts:
        if cv2.contourArea(contour) < 1000:
            continue

        status = 1   
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)  

    status_list.append(status)

    status_list = status_list[-2 : ]  
    if status_list[-1] == 1 and status_list[-2] == 0:
        times.append(datetime.now())
    if status_list[-1] == 0 and status_list[-2] == 1:
        times.append(datetime.now())



    cv2.imshow("Gray Frame", gray)
    cv2.imshow("Delta Frame", delta_frame)
    cv2.imshow("Threshold Frame", thres_frame)
    cv2.imshow("Color Frame", frame)

    key = cv2.waitKey(1)

    if key == ord('q'):
        if status == 1:
            times.append(datetime.now())  
        break
    

print(status_list)
print(times)

for i in range(0, len(times),2):
    df = df.append({"Start":times[i], "End":times[i+1]}, ignore_index = True)

df.to_csv("Times.csv") 

video.release()
cv2.destroyAllWindows()


from bokeh.plotting import figure,show,output_file
from bokeh.models import HoverTool,ColumnDataSource

import cv2, time, pandas

import bokeh
import bokeh.plotting as bp
from bokeh.models import HoverTool
df['Start_string']=df['Start'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['End_string']=df['End'].dt.strftime('%Y-%m-%d %H:%M:%S')
cds = ColumnDataSource(df)

p=bp.figure(x_axis_type="datetime",height=100,width=500,title="Motion Graph")
p.yaxis.minor_tick_line_color=None
p.ygrid[0].ticker.desired_num_ticks=1

hover = HoverTool(tooltips=[('Start','@Start_string'),('End',"@End_string")])
p.add_tools(hover)

q=p.quad(left='Start',right='End',bottom=0,top=1,color='red',source=cds)
output_file('Graph1.html')
show(p)

#import bokeh
#print(bokeh.__version__)
#import bokeh.plotting as bp
#from bokeh.models.tiles import WMTSTileSource

#bp.output_notebook()
#p = bp.figure(tools='pan,wheel_zoom,reset', x_range=(-13884029,  -7453304), y_range=(2698291, 6455972))
#p.add_tile(WMTSTileSource(url="http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png"))

#bp.show(p)

[1, 1]
[datetime.datetime(2020, 6, 10, 2, 56, 41, 1119), datetime.datetime(2020, 6, 10, 2, 57, 5, 437544)]
